Установка необходимых библиотек

In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pymorphy2
from lightgbm import LGBMClassifier

Готовим данные. Поехали!

In [3]:
title_data = pd.read_csv('docs_titles.tsv', sep='\t', encoding='utf-8', lineterminator='\n')
train_groups = pd.read_csv('train_groups.csv')
test_groups = pd.read_csv('test_groups.csv')
sample_subm = pd.read_csv('sample_submission.csv')

In [34]:
doc_to_title = {}
with open('docs_titles.tsv', encoding='utf-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [35]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

Функция для очистки стоп слов.

In [36]:
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
#my_stop = [',', '.', ':', ';', '-', '?', '@', '!', '(',')', '—', '|', '}','{','[',']', '/','\'', 'и', 'а','на','за','в', 'под']
my_stop = [',', '.', ':', ';', '-', '?', '@', '!', '(',')']
print(my_stop)
for i in my_stop:
    stop_words.add(i)

def delete_stop_words(title):
    word_tokens = set(word_tokenize(title))
    filtered_sentence = set()
    for w in word_tokens:
        if w  not in stop_words:
            filtered_sentence.add(w)
    res = filtered_sentence
    return res

[',', '.', ':', ';', '-', '?', '@', '!', '(', ')']


Готовим признаки на ТРЭЙН

In [37]:
y_train = []
X_train = []
groups_train = []
lems_words = []
lems_words_j = []
morph = pymorphy2.MorphAnalyzer()
for new_group in tqdm(traingroups_titledata):
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        all_dist = []
        words = delete_stop_words(title)
        lems_words = []
        for z in words:
            p = morph.parse(z)[0]
            lems_words.append(p.normal_form)
        words = set(lems_words)
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = delete_stop_words(title_j)
            lems_words_j = []
            for z_j in words_j:
                p = morph.parse(z_j)[0]
                lems_words_j.append(p.normal_form)
            words_j = set(lems_words_j)
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:20]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [42:35<00:00, 19.81s/it]

(11690, 20) (11690,)


Признаки by Sanya

In [38]:
y_train = []
X_train_ = []
groups_train = []
lems_words = []
lems_words_j = []
morph = pymorphy2.MorphAnalyzer()
for new_group in tqdm(traingroups_titledata):
    docs = traingroups_titledata[new_group]
    text = []
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        words = delete_stop_words(title)
        lems_words = []
        for z in words:
            p = morph.parse(z)[0]
            lems_words.append(p.normal_form)
        title1 = ''
        for i in lems_words:
            title1 += i
            title1 += ' '
        text.append(title1)
    vectorizer = TfidfVectorizer(ngram_range = (1,3))
    vectorizer.fit(text)
    vector = vectorizer.transform(text)
    vector = vector.toarray()
    for i in range(len(text)):
        res_vector = vector[i]
        res_vector = np.sort(res_vector)
        X_train_.append(res_vector[-20:])
X_train_ = np.array(X_train_)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train_.shape, y_train.shape, groups_train.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:28<00:00,  4.52it/s]

(11690, 20) (11690,) (0,)


In [39]:
y_train = []
X_train_1 = []
groups_train = []
lems_words = []
lems_words_j = []
morph = pymorphy2.MorphAnalyzer()
for new_group in tqdm(traingroups_titledata):
    docs = traingroups_titledata[new_group]
    text = []
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        words = delete_stop_words(title)
        lems_words = []
        for z in words:
            p = morph.parse(z)[0]
            lems_words.append(p.normal_form)
        title1 = ''
        for i in lems_words:
            title1 += i
            title1 += ' '
        text.append(title1)
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    vector = vectorizer.transform(text)
    vector = vector.toarray()
    cosinus = cosine_similarity(vector)
    for i in range(len(text)):
        cosinus_i = cosinus[i]
        res = np.sort(cosinus_i)
        X_train_1.append(res[-20:])
X_train_1 = np.array(X_train_1)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train_1.shape, y_train.shape, groups_train.shape)

  0%|                                                                                          | 0/129 [00:00<?, ?it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['замена подшипник | 21213 ваз ступица нива ', 'ваз сравнить tiu.ru в сочи на оптом потребительский купить цена товар 2107 ', 'ступица тюнинг замена переходный цена ступица калина2 трансмиссия лада купить ', 'классика 21010 21074 ', 'ступица свой рука подшипник замена — нива ', '2110 ваз ', 'полуось подшипник обзор ваз 2121,2123 2101-07 ', 'fag 23 подшипник и страница ступица купить ', 'интернет отзыв 2007 2121 год horsepowers владелец портал автомобильный — ваз нива » ', 'сообщение 33 магазин новость официальный группа из компания вконтакте тольятти и sport торговый ', 'dorognoekam.ru ивеко по передний подшипник инструкция ступица замена дейли через ', 'ступица страница 80 olx.ua ', 'бортжурнал 2160 автокам ╬ проблёмкий год 1994 на — маааленький drive2 ', 'полуось для 

  2%|█▎                                                                                | 2/129 [00:00<00:29,  4.34it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['храм протодиакон мать пришествие кураев второе икона божий — андрей смоленский апокриф ', 'канделаки cтатья д. а. ', 'день какой как т4 когда на правильно и ттг сдавать анализ цикл ', 'родиол жизнь женьшень о розовый или » красиво здоровый образ ', 'грузоподъёмность реконструкция мостовый на перегружатель 32 грузовой тонна атп ', 'posts pragmatika blog ', 'сборный 110-64 всн опора по проектирование фундамент из строительство оболочка и железобетонный бесплатно скачать технический указание мост ', 'из гастрит при что кабачковый приготовить ', 'писать как стих ', 'эпоха в франция ренессанс ', 'грамматика алтай туристический туристский портал алтайский первый ', 'агрегат разработка предприятие автоматизированный элек-тропривод насосный текстильный ', 'философия лекция ка

  3%|██▌                                                                               | 4/129 [00:01<00:32,  3.86it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['в weskachivayu блог запись ', 'как называться скутер в санандрес гта ', 'гта вконтакте 5 сан | скачать ', 'где как джетпак авто по к модель в весь сайт online онлайн официалньный который buffalo плюс пс банить банда случай cкачать поддержка почему от gresley gta официальный высота bravado клонирование для какой какой место nodvd оригинальный скачать пришелец заспавнить бан плюс сайт вступление гта привязать 4 найти ', 'mail.ru city миссия vice gta ответ ', 'gta vc ваенный вертолёт ', 'gta sa мотоцикл управление ', 'карта одежда gta на 4 ', 'пк сайт на игра персональный ', 'как san поменять колесо в andres gta ', "трюк deluxe миссия '' на vc ускорять форум `` gta лодка лодка ", 'код она качивание annihilator ментовский в показать санандрес код старый µtorrent беспредел

  4%|███▏                                                                              | 5/129 [00:01<00:39,  3.18it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['тюль для дом выбирать ', '– сколько штора в пол оставлять фото сантиметр длина ', 'кухонный тюль занавеска видео штора для проект другой на кухня и дизайн фото изделие люверс ', 'обязательно крем для пользователь супер julie sem грудь рост купить дневник ', 'полезно kmg наш заказ пошив знать штора компания штора на форма и салон дизайн штора от длина ', "как ‒ к штора и `` подобрать интерьер '' интерьер интернет-журнал дизайн ", 'няня.р и форум интерьер занавеска дизайн сайт длина ', 'должный штора какой длина быть ', 'должный штора какой длина быть ', 'какой должный штора штора длина быть длина ', 'ok.ru ', 'женский беларусь должный штора сообщество nice.by в форум какой длина женский онлайн » быть ', 'стол видео новогодний и рецепт с фото украшение 2017 ', 'сборка в

  5%|████▍                                                                             | 7/129 [00:02<00:35,  3.41it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['клиника лечение патогенез ринит хронический этиология и форма симптом ', 'ребёнок алоэ побочный у при сок рецепт действие применение насморк ', 'ребёнок из сопля ноздря один почему у ', 'за из кашель высадить сад ', 'mybirds.ru форум аллергия о всё у птица ребёнок ', 'пыльца на цветение лечение аллергия ', 'вылечить как насморк аллергический ', 'свой чем здоровье в инвестировать насморк подсказать » лечить ', 'кто-нибудь гомеопатия помочь ли страница 6 ', 'у лечение аллергический кашель и ребёнок причина ', 'заразиться ребёнок можно на материнство ли > другой от улица ', 'ухо ветеринария форум кошка / насморк и фиолетовый мочь вылечить не ', 'дрожжи диета на при аллергия ', 'избавиться как от средство препарат и первый обзор аллергия помощь оказание народный ', '.. ка

...

 95%|████████████████████████████████████████████████████████████████████████████▎   | 123/129 [00:36<00:01,  3.11it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['википедия — ночь белые ', 'как зимний эрмитаж санкт-петербург история дворец работа добраться часы ', "россия императорский deartravel.ru посмотреть санкт-петербург `` — '' в сад можно ", 'интересный малоизвестный санкт-петербург но в санкт-петербург отдых необычный способ ', 'сеть евразия история ресторан ', 'самый место красивый санкт-петербург ', 'kidsreview.ru пойти в санкт-петербург | с ребёнок куда ', 'достопримечательность поездка санкт-петербург посмотреть что ', 'золотой по треугольник | прогулка санкт-петербург ', 'достопримечательность главный сходить посмотреть июнь-июль на и санкт-петербург туристер.р что куда ', 'достопримечательность | спас-на-кровь санкт-петербург ', 'сходить в санкт-петербург ребёнок с куда ', 'петербург достопримечательность | и санк

 97%|█████████████████████████████████████████████████████████████████████████████▌  | 125/129 [00:37<00:01,  2.88it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['как энергетический вампир от защититься ', 'как энергетический вампир от защититься ', 'энергетический liveinternet как обсуждение такой они российский кто вампир на сервис онлайн-дневник и от защититься ', 'как можно всё магия экстрасенс 2016 энергетический работа на о и мистик-инфо 27 вампир от защититься сентябрь ', 'как от блог в энергетический вампир интересный самый защититься ', 'mail.ru как энергетический вампир от защититься ответ ', 'как энергетический определить вампир и защититься ', 'как непознанный от | и энергетический фэн-шуй вампир школажизни.р защититься ', 'они как жить защититься энергетический с вампир от если ', 'энергетический как такой они кто вампир и от защититься ', 'защититься как энергетический вампиризм ', 'как определить такой защищаться

 98%|██████████████████████████████████████████████████████████████████████████████▊ | 127/129 [00:38<00:00,  2.80it/s]C:\Users\frida\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=['день норма в калория калория сколько ', 'день похудение похудеть диета в для | чтобы калория нужно сколько ', 'день человек в суточный килокалория норма нужно сколько ', 'день похудеть в съедать чтобы калория нужно сколько ', 'похудеть надо чтобы потреблять ediet.ru калория сколько ', 'день норма похудение сколько в для надо употреблять чтобы или худеть калория ', 'есть похудеть калькулятор | чтобы калория нужно сколько ', 'день mail.ru похудеть сколько ккал ответ употреблять в чтобы нужно ', 'день нужно женский в суточный дневник пользователь ninetta23 социальный норма калория сеть myjulia.ru потребление дневник сколько ', 'день похудеть в съедать чтобы калория сколько ', 'день похудеть в addhealth.ru съедать | чтобы калория нужно сколько ', 'день калькулятор в | кал

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:38<00:00,  3.34it/s]

(11690, 20) (11690,) (0,)


In [40]:
X_max = np.hstack((X_train,X_train_,X_train_1)) ##объединяем признаки мои и сани
X_max.shape

(11690, 60)

ВАЛИДАЦИЯ

In [41]:
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_max, y_train, test_size=0.35, shuffle=False)#, random_state=0, stratify=y_train)

In [43]:
reg = []
for learning_rate in tqdm([0.001,0.002,0.003,0.004,0.005,0.006,0.01]): #, 0.3, 0.4, 0.5):
    for n_estimators in([2000,2500,3000,3500,4000,4500,5000,5500]):
        for max_depth in (5,7,10,13,15,18,21,25):
            clf = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth,subsample=0.75)
            clf.fit(X_train_val, y_train_val)
            y_pred = clf.predict(X_test_val)
            f1 = f1_score(y_test_val, y_pred)
            reg.append((learning_rate, n_estimators, max_depth, f1))
            print((learning_rate, n_estimators, max_depth, f1))

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

(0.001, 2000, 5, 0.7142254115660616)
(0.001, 2000, 7, 0.7309136420525657)
(0.001, 2000, 10, 0.7336390162567737)
(0.001, 2000, 13, 0.7328624844204404)
(0.001, 2000, 15, 0.7331670822942643)
(0.001, 2000, 18, 0.7325581395348838)
(0.001, 2000, 21, 0.7336099585062241)
(0.001, 2000, 25, 0.7336099585062241)
(0.001, 2500, 5, 0.7252289758534554)
(0.001, 2500, 7, 0.7398273736128236)
(0.001, 2500, 10, 0.7412874128741288)
(0.001, 2500, 13, 0.7399507793273175)
(0.001, 2500, 15, 0.7403451109285127)
(0.001, 2500, 18, 0.7418957734919983)
(0.001, 2500, 21, 0.7408624229979466)
(0.001, 2500, 25, 0.7408624229979466)
(0.001, 3000, 5, 0.7292269532864821)
(0.001, 3000, 7, 0.74007367990176)
(0.001, 3000, 10, 0.741042345276873)
(0.001, 3000, 13, 0.7416734362307067)
(0.001, 3000, 15, 0.740228013029316)
(0.001, 3000, 18, 0.7414634146341463)
(0.001, 3000, 21, 0.7436001625355546)
(0.001, 3000, 25, 0.7430894308943089)
(0.001, 3500, 5, 0.7300411522633744)
(0.001, 3500, 7, 0.7384740922072623)
(0.001, 3500, 10, 0.7417

 14%|███████████▌                                                                     | 1/7 [15:42<1:34:14, 942.41s/it]

(0.001, 6000, 25, 0.7411858974358974)
(0.002, 2000, 5, 0.728542094455852)
(0.002, 2000, 7, 0.741190765492102)
(0.002, 2000, 10, 0.7434343434343433)
(0.002, 2000, 13, 0.7404117884537748)
(0.002, 2000, 15, 0.7408303103587263)
(0.002, 2000, 18, 0.742834073475979)
(0.002, 2000, 21, 0.7444489301574485)
(0.002, 2000, 25, 0.7439418416801293)
(0.002, 2500, 5, 0.7295081967213115)
(0.002, 2500, 7, 0.739516129032258)
(0.002, 2500, 10, 0.7427652733118971)
(0.002, 2500, 13, 0.7393403057119872)
(0.002, 2500, 15, 0.7433628318584071)
(0.002, 2500, 18, 0.7428571428571428)
(0.002, 2500, 21, 0.7402649538338016)
(0.002, 2500, 25, 0.7388329979879276)
(0.002, 3000, 5, 0.7298850574712643)
(0.002, 3000, 7, 0.7387459807073956)
(0.002, 3000, 10, 0.7434973989595839)
(0.002, 3000, 13, 0.7416900280336404)
(0.002, 3000, 15, 0.7430861723446893)
(0.002, 3000, 18, 0.7393403057119872)
(0.002, 3000, 21, 0.7412755716004814)
(0.002, 3000, 25, 0.7393744987971129)
(0.002, 3500, 5, 0.7332789559543229)
(0.002, 3500, 7, 0.7404

 29%|███████████████████████▏                                                         | 2/7 [29:54<1:16:16, 915.32s/it]

...

(0.005, 6000, 25, 0.7329884600079587)
(0.006, 2000, 5, 0.7338187702265372)
(0.006, 2000, 7, 0.7413034786085565)
(0.006, 2000, 10, 0.7390612569610183)
(0.006, 2000, 13, 0.7452380952380951)
(0.006, 2000, 15, 0.7457222443294867)
(0.006, 2000, 18, 0.7420382165605095)
(0.006, 2000, 21, 0.7453317441398492)
(0.006, 2000, 25, 0.7438540840602695)
(0.006, 2500, 5, 0.7340339531123686)
(0.006, 2500, 7, 0.7415101877746704)
(0.006, 2500, 10, 0.7451292246520875)
(0.006, 2500, 13, 0.7435592548553309)
(0.006, 2500, 15, 0.7445371473976956)
(0.006, 2500, 18, 0.7471218737594283)
(0.006, 2500, 21, 0.7432432432432431)
(0.006, 2500, 25, 0.745253164556962)
(0.006, 3000, 5, 0.7359098228663447)
(0.006, 3000, 7, 0.7372643401524268)
(0.006, 3000, 10, 0.7428343949044587)
(0.006, 3000, 13, 0.7448330683624801)
(0.006, 3000, 15, 0.7416534181240064)
(0.006, 3000, 18, 0.7459373761395165)
(0.006, 3000, 21, 0.7401824672748909)
(0.006, 3000, 25, 0.7449424831416104)
(0.006, 3500, 5, 0.7342685370741483)
(0.006, 3500, 7, 0.7

 86%|█████████████████████████████████████████████████████████████████████▍           | 6/7 [1:24:13<14:00, 840.79s/it]

(0.006, 6000, 25, 0.7395091053048298)
(0.01, 2000, 5, 0.7340553549939831)
(0.01, 2000, 7, 0.736)
(0.01, 2000, 10, 0.7372611464968152)
(0.01, 2000, 13, 0.7373857767183154)
(0.01, 2000, 15, 0.7360890302066774)
(0.01, 2000, 18, 0.7387673956262425)
(0.01, 2000, 21, 0.7346123101518784)
(0.01, 2000, 25, 0.7436507936507936)
(0.01, 2500, 5, 0.7329842931937173)
(0.01, 2500, 7, 0.7328548644338119)
(0.01, 2500, 10, 0.7396166134185304)
(0.01, 2500, 13, 0.7332007952286282)
(0.01, 2500, 15, 0.7360446570972887)
(0.01, 2500, 18, 0.7383512544802867)
(0.01, 2500, 21, 0.7310454908220271)
(0.01, 2500, 25, 0.7377636291285317)
(0.01, 3000, 5, 0.7317466720451794)
(0.01, 3000, 7, 0.7298050139275766)
(0.01, 3000, 10, 0.7382282521947326)
(0.01, 3000, 13, 0.73409719478467)
(0.01, 3000, 15, 0.7352472089314196)
(0.01, 3000, 18, 0.7396496815286623)
(0.01, 3000, 21, 0.7332007952286282)
(0.01, 3000, 25, 0.7380573248407644)
(0.01, 3500, 5, 0.7322580645161291)
(0.01, 3500, 7, 0.7294398092967818)
(0.01, 3500, 10, 0.7365

100%|█████████████████████████████████████████████████████████████████████████████████| 7/7 [1:37:55<00:00, 839.41s/it]

(0.01, 6000, 25, 0.7338869118228548)


In [46]:
max_score = []
for i in reg:
    max_score.append(i[3])
max_sxore = np.asarray(max_score)
max(max_score)

0.7477155343663091

Подготовка тестовых данных

In [ ]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [ ]:
y_test = []
X_test = []
groups_test = []
lems_words = []
lems_words_j = []
for new_group in tqdm(testgroups_titledata):
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = delete_stop_words(title)
        lems_words = []
        for z in words:
            p = morph.parse(z)[0]
            lems_words.append(p.normal_form)
        words = set(lems_words)
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = delete_stop_words(title_j)
            lems_words_j = []
            for z_j in words_j:
                p = morph.parse(z_j)[0]
                lems_words_j.append(p.normal_form)
            words_j = set(lems_words_j)
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:20])
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape,groups_test.shape)

In [ ]:
X_test_ = []
groups_test = []
lems_words = []
lems_words_j = []
for new_group in tqdm(testgroups_titledata):
    docs = testgroups_titledata[new_group]
    text = []
    for k, (doc_id, title) in enumerate(docs):
        words = delete_stop_words(title)
        lems_words = []
        for z in words:
            p = morph.parse(z)[0]
            lems_words.append(p.normal_form)
        title1 = ''
        for i in lems_words:
            title1 += i
            title1 += ' '
        text.append(title1)
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    vectorizer.fit(text)
    vector = vectorizer.transform(text)
    vector = vector.toarray()
    for i in range(len(text)):
        res_vector = vector[i]
        res_vector = np.sort(res_vector)
        X_test_.append(res_vector[-20:])
X_test_ = np.array(X_test_)
groups_test = np.array(groups_test)
print (X_test.shape,groups_test.shape)

In [ ]:
X_max_test = np.hstack((X_test,X_test_))

Делаем модель с лучшими параметрами и погнали предиктить

In [ ]:
X_max_test = np.hstack((X_test,X_test_))

In [ ]:
sub = clf.predict(X_max_test)

In [ ]:
sample_subm.target = sub

In [ ]:
sub.shape

In [ ]:
sample_subm.to_csv('submit5.csv', index = False)